In [1]:
!pip install sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.7/406.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━

In [1]:
import sagemaker

import boto3

import pandas as pd



# Initialize SageMaker session

sagemaker_session = sagemaker.Session()



# Get the SageMaker execution role

role = sagemaker.get_execution_role()



# S3 bucket for storing data

bucket = sagemaker_session.default_bucket()

prefix = "nlp-model-demo"






/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:11                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/session.py:265 in __init__                     │
│                                                                                                  │
│    262 │   │   self.lambda_client = None                                                         │
│    263 │   │   self.settings = settings if settings else SessionSettings()                       │
│    264 │   │                                                                                     │
│ ❱  265 │   │   self._initialize(                                                                 │
│    266 │   │   │   boto_session=boto_session,                                                    │
│    267 │   │   │   sagemaker_client=sagemaker_client,                                            │
│    268 │   │   │   sagemaker_runtime_client=sagemaker_runtime_client,                            │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/session.py:293 in _initialize                  │
│                                                                                                  │
│    290 │   │                                                                                     │
│    291 │   │   self._region_name = self.boto_session.region_name                                 │
│    292 │   │   if self._region_name is None:                                                     │
│ ❱  293 │   │   │   raise ValueError(                                                             │
│    294 │   │   │   │   "Must setup local AWS configuration with a region supported by SageMaker  │
│    295 │   │   │   )                                                                             │
│    296                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [2]:
def train():

    # Argument parser for SageMaker input

    parser = argparse.ArgumentParser()

    parser.add_argument("--train_data", type=str, default=os.environ["SM_CHANNEL_TRAIN"])

    args = parser.parse_args()



    # Load dataset from the provided path

    train_data_path = os.path.join(args.train_data, "processed_reviews.csv")

    df = pd.read_csv(train_data_path)



    # Split data

    X = df["Text"]

    y = df["Sentiment"]
# Create a text-processing pipeline

    pipeline = Pipeline([

        ("tfidf", TfidfVectorizer(stop_words="english")),

        ("clf", LogisticRegression())

    ])



    # Train model

    pipeline.fit(X, y)



    # Save trained model

    model_path = os.path.join("/opt/ml/model", "model.joblib")

    joblib.dump(pipeline, model_path)

    print("Model saved at", model_path)



if _name_ == "_main_":

    train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:54                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name '_name_' is not defined

In [ ]:
from sagemaker.sklearn.estimator import SKLearn



# Define SageMaker SKLearn Estimator

sklearn_estimator = SKLearn(

    entry_point="train.py",

    framework_version="0.23-1",

    instance_type="ml.m5.large",

    role=role,

    sagemaker_session=sagemaker_session,

)

# Train the model on SageMaker

sklearn_estimator.fit({"train": s3_train_data})

In [4]:
%%writefile inference.py

import joblib

import os

import json

import pandas as pd

# Load trained model

def model_fn(model_dir):

    model_path = os.path.join(model_dir, "model.joblib")

    return joblib.load(model_path)

# Parse input JSON

def input_fn(request_body, request_content_type):

    if request_content_type == "application/json":

        data = json.loads(request_body)

        return pd.DataFrame(data, columns=["Text"])

    else:

        raise ValueError("Unsupported content type: {}".format(request_content_type))

# Generate predictions

def predict_fn(input_data, model):

    return model.predict(input_data["Text"]).tolist()






Writing inference.py


In [5]:
from sagemaker.sklearn.model import SKLearnModel



# Get model path from training job

model_data = sklearn_estimator.model_data



# Create a SageMaker model

sklearn_model = SKLearnModel(

    model_data=model_data,

    role=role,

    entry_point="inference.py",

    framework_version="0.23-1",

    sagemaker_session=sagemaker_session,

)



# Deploy the model to a real-time endpoint

predictor = sklearn_model.deploy(instance_type="ml.m5.large", initial_instance_count=1)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:7                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'sklearn_estimator' is not defined

In [7]:
import json
test_data = json.dumps(["This product is amazing!", "Worst product ever."])
response = predictor.predict(test_data)
print("Predictions:", response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:3                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predictor' is not defined